Importing the Libraries


In [1]:
import numpy as np
import pandas as pd
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import warnings
import csv


Supressing the warnings

In [2]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

Reading the dataset

In [3]:
Data_path = '../Data/Training.csv'
data = pd.read_csv(Data_path).dropna(axis=1)

Encoding

In [4]:
encoder = LabelEncoder()
data['prognosis'] = encoder.fit_transform(data['prognosis'])
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]


In [5]:
Y

0       15
1       15
2       15
3       15
4       15
        ..
4915     0
4916     2
4917    38
4918    35
4919    27
Name: prognosis, Length: 4920, dtype: int64

Building the models

In [6]:
svm_model = SVC()
svm_model.fit(X, Y)

nb_model = GaussianNB()
nb_model.fit(X, Y)

rf_model = RandomForestClassifier()
rf_model.fit(X, Y)


RandomForestClassifier()

Functions for predicting Disease

In [7]:
symptoms = X.columns.values

symptom_index = {}
for index, value in enumerate(symptoms):
    symptom= " ".join([i.capitalize() for i in value.split("_")])
    symptom_index[symptom] = index

data_dict = {
    "symptom_index" : symptom_index,
    "predictions_classes" : encoder.classes_ 
}


# print("Symptom Index Dictionary:", data_dict["symptom_index"])

def predictDisease(symptoms):
    symptoms = symptoms.split(',')

    input_data = [0]*len(data_dict["symptom_index"])
    for symptom in symptoms:
        index = data_dict["symptom_index"][symptom]
        input_data[index] = 1

        input_data = np.array(input_data).reshape(1,-1)
        
        rf_prediction = data_dict["predictions_classes"][rf_model.predict(input_data)[0]]
        nb_prediction = data_dict["predictions_classes"][nb_model.predict(input_data)[0]]
        svm_prediction = data_dict["predictions_classes"][svm_model.predict(input_data)[0]]

        final_prediction = mode([rf_prediction, nb_prediction, svm_prediction], axis=0,keepdims=True,  nan_policy='omit')[0][0]
        predictions = {
            "rf_model_prediction" :rf_prediction,
            "nb_model_prediction" :nb_prediction,
            "svm_model_prediction" :svm_prediction,
            "final_prediction" : final_prediction
        }
        return predictions
        


Taking input(symptoms)

In [8]:
Input_Symptoms=input('Enter the Diseases:')
disease = predictDisease(Input_Symptoms)
# print(disease);
final_disease=disease["final_prediction"]
print("Maybe You are having: ", final_disease);
print("Your Symptoms: ", Input_Symptoms);

Maybe You are having:  Paralysis (brain hemorrhage)
Your Symptoms:  Headache


In [9]:
disease_descriptions={}
predicted_disease = final_disease

with open('../Data/symptom_Description.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row 

    for row in reader:
        disease_name, description = row  
        disease_descriptions[disease_name] = description

if predicted_disease in disease_descriptions:
    description = disease_descriptions[predicted_disease]
    print(f"Description of {predicted_disease}: {description}")
else:
    print(f"Description for {predicted_disease} not found.")

Description of Paralysis (brain hemorrhage): Intracerebral hemorrhage (ICH) is when blood suddenly bursts into brain tissue, causing damage to your brain. Symptoms usually appear suddenly during ICH. They include headache, weakness, confusion, and paralysis, particularly on one side of your body.


In [11]:
disease_precautions={}
with open('../Data/symptom_precaution.csv','r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)

    for row in reader:
        disease,precaution_1, precaution_2, precaution_3, precaution_4 = row
        disease_precautions[disease]= [precaution_1, precaution_2, precaution_3, precaution_4]

    if predicted_disease in disease_precautions:
         precautions = disease_precautions[predicted_disease]
         print(f"Precautions for {predicted_disease}:")
         for i, precaution in enumerate(precautions):
            if precaution == '':
                continue
            print(f"{precaution.capitalize()}")
    else:
        print(f"Precautions for {predicted_disease} not found.")

Precautions for Paralysis (brain hemorrhage) not found.


In [ ]:
import pickle
file_name = 'disease_prediction_model.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(final_disease, f)